In [2]:
import re
from typing import List
import os
import pyspark.sql as sql
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import DoubleType, IntegerType, ArrayType, StringType
from pyspark.sql.functions import udf, explode, rank, desc, col, max, sum, lower, countDistinct, regexp_extract, to_timestamp


In [3]:
spark = SparkSession \
    .builder \
    .appName("L2_reports_with_apache_spark") \
    .config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.13.0")\
    .getOrCreate()

23/12/06 23:02:56 WARN Utils: Your hostname, vasser232-desktop resolves to a loopback address: 127.0.1.1; using 192.168.1.116 instead (on interface enp6s0)
23/12/06 23:02:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/vasser232/%d0%97%d0%b0%d0%b3%d1%80%d1%83%d0%b7%d0%ba%d0%b8/spark-3.5.0-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/vasser232/.ivy2/cache
The jars for the packages stored in: /home/vasser232/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3b785eb3-6752-4a6e-ba79-78a85efc3784;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.13.0 in central
	found commons-io#commons-io;2.8.0 in central
	found org.glassfish.jaxb#txw2;2.3.4 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.2.5 in central
:: resolution report :: resolve 215ms :: artifacts dl 10ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.13.0 from central in [default]
	commons-io#commons-io;2.8.0 from central in [default]
	org.apache.ws.xmlschema#xmlschema-core;2.2.5 from central in [default]
	org.glassfish.jaxb#txw2;2.3.4 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | nu

In [4]:
spark.version

'3.5.0'

## Загрузка данных

### Указание путей с файлами datasets

In [5]:
posts_path = os.path.join("posts_sample.xml")
prog_lang_path = os.path.join("programming-languages.csv")

### Чтение данных о постах Stack Overflow

In [6]:
posts_data = spark.read.format("xml").options(rowTag="row").load(posts_path)
print("Posts")
posts_data.printSchema()

Posts
root
 |-- _AcceptedAnswerId: long (nullable = true)
 |-- _AnswerCount: long (nullable = true)
 |-- _Body: string (nullable = true)
 |-- _ClosedDate: timestamp (nullable = true)
 |-- _CommentCount: long (nullable = true)
 |-- _CommunityOwnedDate: timestamp (nullable = true)
 |-- _CreationDate: timestamp (nullable = true)
 |-- _FavoriteCount: long (nullable = true)
 |-- _Id: long (nullable = true)
 |-- _LastActivityDate: timestamp (nullable = true)
 |-- _LastEditDate: timestamp (nullable = true)
 |-- _LastEditorDisplayName: string (nullable = true)
 |-- _LastEditorUserId: long (nullable = true)
 |-- _OwnerDisplayName: string (nullable = true)
 |-- _OwnerUserId: long (nullable = true)
 |-- _ParentId: long (nullable = true)
 |-- _PostTypeId: long (nullable = true)
 |-- _Score: long (nullable = true)
 |-- _Tags: string (nullable = true)
 |-- _Title: string (nullable = true)
 |-- _ViewCount: long (nullable = true)



In [7]:
prog_lang_data = (
    spark.read.option("header", True)
    .option("inferSchema", True)
    .option("timestampFormat", "M/d/y H:m")
    .csv(prog_lang_path)
)

print("programming-languages")
prog_lang_data.printSchema()

programming-languages
root
 |-- name: string (nullable = true)
 |-- wikipedia_url: string (nullable = true)



## Работа с данными

### Выбор необходимых столбцов

In [8]:
def get_tags(tags_string):
    if tags_string is None:
        return []
    
    pattern = r'<(.+?)>'
    tags = re.findall(pattern, tags_string)
    
    return tags

def get_year(date_and_time):
    return date_and_time.year

get_tags_udf = udf(get_tags, ArrayType(StringType()))
get_year_udf = udf(get_year, IntegerType())

In [9]:
# Применение функций к столбцам DataFrame
posts_data_simplified = posts_data.withColumn(
    "tags", get_tags_udf(col("_Tags"))
).withColumn("year", get_year_udf(col("_LastActivityDate")))

# Выбор нужных столбцов
posts_data_simplified = posts_data_simplified.select(
    col("tags"), col("year"), col("_ViewCount").alias("views")
)

# Отображение полученных данных
posts_data_simplified.show()

+--------------------+----+------+
|                tags|year| views|
+--------------------+----+------+
|[c#, floating-poi...|2019| 42817|
|[html, css, inter...|2019| 18214|
|                  []|2017|  NULL|
|[c#, .net, datetime]|2019|555183|
|[c#, datetime, ti...|2019|149445|
|                  []|2018|  NULL|
|[html, browser, t...|2019|176405|
|        [.net, math]|2018|123231|
|                  []|2010|  NULL|
|                  []|2010|  NULL|
|                  []|2010|  NULL|
|                  []|2010|  NULL|
|                  []|2010|  NULL|
|                  []|2010|  NULL|
|                  []|2010|  NULL|
|                  []|2010|  NULL|
|                  []|2010|  NULL|
|                  []|2010|  NULL|
|                  []|2013|  NULL|
|                  []|2010|  NULL|
+--------------------+----+------+
only showing top 20 rows



### Удаление тегов, не соответствующих языкам программирования

In [10]:
# Преобразование названий языков программирования в нижний регистр
prog_lang_data_modified = prog_lang_data.withColumn("tag_lowercase", lower(prog_lang_data["name"]))

# Разделение массива тегов на отдельные строки
posts_data_exploded = posts_data_simplified.select("year", explode("tags").alias("tag"), "views")

# Присоединение таблицы языков программирования для фильтрации постов по языку
posts_data_filtered = posts_data_exploded.join(
    prog_lang_data_modified,
    (posts_data_exploded["tag"] == prog_lang_data_modified["tag_lowercase"]),
    "inner"
)

# Выбор нужных столбцов
posts_data_filtered = posts_data_filtered.select("year", "tag", "views")

# Отображение результата
posts_data_filtered.show()

+----+-----------+-----+
|year|        tag|views|
+----+-----------+-----+
|2010|       java|  132|
|2010|        php| 1258|
|2015|       ruby| 9649|
|2010|          c| 2384|
|2015|        php| 1987|
|2010|     python| 3321|
|2010| javascript|  128|
|2010|applescript|  477|
|2010|        php| 1748|
|2010|        php|  998|
|2013| javascript| 2095|
|2010|        sed|  447|
|2015|     python| 6558|
|2015|       java|  214|
|2015|       ruby|  214|
|2013|objective-c|  852|
|2010| javascript|  179|
|2010|          r| 6709|
|2010|        php|   78|
|2010| javascript| 1280|
+----+-----------+-----+
only showing top 20 rows



### Подготовка к отчету

In [11]:
# Группировка по году и тегам, с суммированием просмотров
grouped_data = posts_data_filtered.groupBy("year", "tag").agg(sum("views").alias("total_views"))

# Сортировка по году и количеству просмотров
sorted_data = grouped_data.orderBy("year", col("total_views").desc())

# Отображение результата
sorted_data.show()

+----+------------+-----------+
|year|         tag|total_views|
+----+------------+-----------+
|2008|        java|      11532|
|2008|        ruby|       1843|
|2008|         x++|       1363|
|2009|      python|      32219|
|2009|  javascript|      17139|
|2009|           c|      16356|
|2009|        java|      13533|
|2009|         php|      12876|
|2009|        bash|       4410|
|2009|     haskell|       3992|
|2009|       xpath|       3869|
|2009| objective-c|       3671|
|2009|      delphi|       3477|
|2009|        ruby|       2844|
|2009|  powershell|        536|
|2009|actionscript|        318|
|2010|        java|      53333|
|2010|      matlab|      51865|
|2010| objective-c|      43878|
|2010|         php|      39730|
+----+------------+-----------+
only showing top 20 rows



### Итоговый отчет (N самых популярных языков программирования за год)

In [22]:
from pyspark.sql.functions import col

# Группировка и суммирование просмотров по году и тегам
posts_data_sorted = posts_data_filtered.groupBy("year", "tag").agg(sum("views").alias("total_views"))

# Сортировка по году и общему количеству просмотров
posts_data_sorted = posts_data_sorted.orderBy("year", desc("total_views"))

# Разбиение по годам и сохранение в отдельные файлы
languages_per_year = 10

# Перебор по годам для сохранения данных в отдельные файлы
for year in posts_data_sorted.select("year").distinct().collect():
    year = year["year"]
    year_data = posts_data_sorted.filter(col("year") == year)

    # Определение пути для сохранения данных по текущему году
    output_path_year = f"/media/vasser232/System/output/{year}"
    
    # Разбиение по году и сохранение в формате Parquet
    year_data.write.partitionBy("year").format("parquet").save(output_path_year)


In [24]:
import pandas as pd
df = pd.read_parquet("/media/vasser232/System/output/2009/year=2009/part-00000-0f424b57-b3a6-4106-ade6-40c602a2e3ac.c000.snappy.parquet")
df

,tag,total_views
0,python,32219
1,javascript,17139
2,c,16356
3,java,13533
4,php,12876
5,bash,4410
6,haskell,3992
7,xpath,3869
8,objective-c,3671
9,delphi,3477
